In [2]:
from IPython.display import HTML
import numpy as np
import sys
sys.path.insert(0, 'other/')
import pandas as pd
import datetime
import os
from ipywidgets import FloatProgress,FloatText
from IPython.display import display
import time
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
import gc
import cProfile
from tqdm import tqdm_notebook
%matplotlib inline
#make wider graphs
sns.set(rc={'figure.figsize':(12,5)})
plt.figure(figsize=(12,5))
path='../data/LHC/train/'
out_path='../data/LHC/clustered/'

from functions.other import calc_features, get_event, score_event_fast, load_obj
from functions.expand import *
from functions.cluster import *
from functions.ml_model import merge_with_probabilities,precision_and_recall,get_features,get_predictions

# the following two lines are for changing imported functions, and not needing to restart kernel to use their updated version
%load_ext autoreload
%autoreload 2



<Figure size 864x360 with 0 Axes>

### Clustering
Define parameters and run clustering, twice:

In [3]:
weights={'pi':1,'theta':0.15}
stds={'z0':7.5, 'kt':7.5e-4}
d =    {'sint':[225,110,110],
        'cost':[225,110,110],
          'phi':[550,260,260],
        'min_group':[11,11,10],
        'npoints':[250,1250,500]}
filters=pd.DataFrame(d)
nu=250

def run_clustering(i):
    event_num=1000+i
    event_prefix = 'event00000{}'.format(event_num)
    print('running '+event_prefix)
    hits, cells, particles, truth = get_event(path,event_prefix)
    history=[]

    resa1=clustering(hits,stds,filters,phik=3.3,nu=nu)
    resa1["event_id"]=event_num-1000
    score = score_event_fast(truth, resa1.rename(index=str, columns={"label": "track_id"}))
    print("Your score: ", score)
    resa1.to_csv("{}{}_cluster.csv".format(out_path,event_prefix))


In [4]:
### RUN PARALLEL 

from joblib import Parallel, delayed
import multiprocessing
from tqdm import tqdm

num_cores = multiprocessing.cpu_count()
res_list = Parallel(n_jobs=num_cores)(delayed(run_clustering)(i) for i in tqdm(range(100)))


100%|██████████████████████████████████████████████████████████████████████████████| 100/100 [1:14:11<00:00, 44.52s/it]
